In [1]:
import parametros as par
import utils
import dask.dataframe as dd
from dask.distributed import Client
import time
import asyncio
import pandas as pd
from loguru import logger
#from pandera.typing.dask import DataFrame, Series
#import pandera as pa
from sqlalchemy import table, column, select, types, Float
from sqlalchemy.dialects import oracle
from datetime import datetime 
import sqlalchemy
import sys

In [2]:
client= Client("tcp://10.128.0.48:8786")

In [3]:
p=par.ParamApdw()

In [23]:
data=p.arquivo_origem

In [5]:
url_db = p.url_db

In [6]:
timestamp=datetime.today().strftime('%Y-%m-%d_%H:%M')
logger.add(f'out_{timestamp}.log', level="TRACE", serialize=True, backtrace= True, diagnose=True)

1

In [7]:
def read_oracle(colunas=None,meta=None):
    #le  a tabela
   try:
        logger.info("Iniciando leitura do Oracle")
        if (colunas is None) & (meta is None) :
            df = dd.read_sql_table(table_name='tb_raw_data', con=url_db,index_col= 'cod', dtype={
                k.lower(): p.schema_base[k] for k in p.schema_base 
                }).reset_index()
            #erro abaixo sucess
            logger.success("Leitura de toda a tabela concluida.")
            return df
        elif meta is not None:
            df = dd.read_sql_table(table_name='tb_raw_data', con=url_db,index_col= 'cod', meta= p.meta_schema)
            logger.success("Leitura dos Metadados concluida.")
            return df
        else:
            df = dd.read_sql_table(table_name='tb_raw_data', columns=colunas, con=url_db,index_col= 'cod', dtype={
                k.lower(): p.schema_base[k] for k in p.schema_base if k.lower() in colunas
                }).reset_index()
            logger.success("Leitura das colunas concluida.")
            return df
   except sqlalchemy.exc.NoSuchTableError as e:
        logger.error(f"Erro na leitura do Oracle: {e}")
        

LEITURA BRUTA DE DADOS DO DIRETORIO
====================================================================================

In [33]:
def read_raw_data(data,timestamp_ini):
    #le os dados
    
    try:
        logger.info("Lendo dados")
        df=dd.read_csv(data, sep="|", assume_missing=True).astype(dtype=p.raw_schema_base)
        logger.success("Leitura concluida")
        return df
    except Exception as e:
        logger.error(f"Erro na leitura do arquivo:{e}")
        utils.update_log_fim(nome_processo='Pipeline_Raw',timestamp_ini=timestamp_ini,status='Falha',motivo_erro=f'{e.__class__.__name__}: {e}')
        sys.exit(1)

    

In [34]:
def pipeline_raw_oracle(df,timestamp_ini):    
    try:
        start_time = time.time()
        logger.info("Iniciando Inserção no Oracle")
        df_result=(
            df.drop('Unnamed: 11', axis=1)
              .assign(DAT_ULT_AT=dd.to_datetime(df['DAT_ULT_AT'],format='%Y-%m-%d',errors='coerce'))
              .to_sql("tb_raw_data",url_db, if_exists='replace', index=False,  
                      compute=True, parallel=True, chunksize=500,dtype={
                          k: p.oracle_types[v] for (k,v) in p.schema_base.items() if k in (p.raw_schema_base.keys())
                      }
                     )            
        )
        end_time = time.time()
        elapsed_time = end_time - start_time
        logger.success(f"Inserção concluida, {round(elapsed_time,2)} segs ")
        utils.update_log_fim(nome_processo='Pipeline_Raw',timestamp_ini=timestamp_ini,status='Sucesso')
    except Exception as e:
        logger.error("Erro no envio bruto para o Oracle")
        utils.update_log_fim(nome_processo='Pipeline_Raw',timestamp_ini=timestamp_ini,status='Falha',motivo_erro=f'{e.__class__.__name__}:{e}')


In [35]:
timestamp_ini=datetime.today()
utils.insere_log_inicio(nome_processo='Pipeline_Raw',timestamp_ini=timestamp_ini)
df=read_raw_data(data,timestamp_ini)
pipeline_raw_oracle(df,timestamp_ini)

2023-09-14 18:36:13.178 | INFO     | __main__:read_raw_data:5 - Lendo dados
2023-09-14 18:36:13.181 | ERROR    | __main__:read_raw_data:10 - Erro na leitura do arquivo:An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: [Errno 21] Is a directory: '/home/thiago_lemos/poc_dask_app'


AttributeError: 'tuple' object has no attribute 'tb_frame'

2023-09-14 19:46:03,864 - distributed.batched - INFO - Batched Comm Closed <TCP (closed) Client->Scheduler local=tcp://10.128.0.48:60874 remote=tcp://10.128.0.48:8786>
Traceback (most recent call last):
  File "/root/anaconda3/envs/poc_dask/lib/python3.11/site-packages/distributed/batched.py", line 115, in _background_send
    nbytes = yield coro
             ^^^^^^^^^^
  File "/root/anaconda3/envs/poc_dask/lib/python3.11/site-packages/tornado/gen.py", line 769, in run
    value = future.result()
            ^^^^^^^^^^^^^^^
  File "/root/anaconda3/envs/poc_dask/lib/python3.11/site-packages/distributed/comm/tcp.py", line 269, in write
    raise CommClosedError()
distributed.comm.core.CommClosedError


FUNÇÃO RELATORIO DE VALORES FEVEREIRO
====================================================================================

In [32]:
def soma_valores_fevereiro(df,colunas):
    #Aplica as regras de negocio
    try:
        start_time = time.time()
        logger.info("Iniciando Processo SomaValoresFevereiro")
        df=df.drop_duplicates()
        df_result = (df.assign(dat_ult_at=dd.to_datetime(df['dat_ult_at'],format='%Y-%m-%d',errors='coerce'))
                    .query("cod==1 & dat_ult_at.between('2023-02-01','2023-02-28')")
                    .groupby(['cod','dat_ult_at'])['vlr_doc_seg_emit','vlr_premio_cobra','vlr_sinistro_avi','vlr_sinistro_pag'].sum()
                    .to_sql("tb_soma_valores_fevereiro",url_db, if_exists='replace', index=True,  compute=True, parallel=True, chunksize=500, dtype={
                        k.lower(): p.oracle_types[v] for (k,v) in p.schema_base.items() if k.lower() in colunas
                    }
           ))
        end_time = time.time()
        elapsed_time = end_time - start_time
        logger.success(f"Inserção concluida,{round(elapsed_time,2)} ")
    except:
        logger.error(f"Erro na task SomaValoresFevereiro" )

In [37]:
#%%timeit
colunas= ['cod','dat_ult_at','vlr_doc_seg_emit','vlr_premio_cobra','vlr_sinistro_avi','vlr_sinistro_pag']
df=read_oracle(colunas)
soma_valores_fevereiro(df,colunas)

2023-09-11 13:06:32.071 | INFO     | __main__:read_oracle:4 - Iniciando leitura do Oracle
2023-09-11 13:06:32.158 | ERROR    | __main__:read_oracle:23 - Erro na leitura do Oracle: tb_raw_data
2023-09-11 13:06:32.160 | INFO     | __main__:soma_valores_fevereiro:5 - Iniciando Processo SomaValoresFevereiro
2023-09-11 13:06:32.162 | ERROR    | __main__:soma_valores_fevereiro:18 - Erro na task SomaValoresFevereiro


FUNÇÃO CONSULTA TABELA DE REFERÊNCIA
====================================================================================

In [132]:
def consulta_tab_ref(df, colunas):
    try:
        start_time = time.time()
        logger.info("Iniciando Processo ConsultaTabRef")
        df_ref = dd.read_sql(sql='table_ref', con=url_db,index_col= 'cod')
        df_merge= dd.merge(df,df_ref, how='left', left_index = True, right_index = True)
        df_merge=(df_merge.fillna({
            'nome_cliente': "Cliente não encontrado"
        }).to_sql("tb_consulta_ref",url_db, if_exists='replace', index=True,  compute=True, parallel=True, chunksize=500,dtype={
                        k.lower(): p.oracle_types[v] for (k,v) in p.schema_base.items() if k.lower() in colunas
                    }))
        end_time = time.time()
        elapsed_time = end_time - start_time
        logger.success(f"Inserção concluida,{round(elapsed_time,2)} ")
    except: 
        logger.error(f"Erro na task ConsultaTabRef" )


In [133]:
#%%timeit 
colunas= ['cod','dat_ult_at','vlr_doc_seg_emit','vlr_premio_cobra','vlr_sinistro_avi','vlr_sinistro_pag']
df=read_oracle(colunas)
consulta_tab_ref(df,colunas)


FUNÇÃO SEPARA COD RA
====================================================================================


In [134]:
def filtra_cod(cod):
    try:
        start_time = time.time()
        logger.info(f"Iniciando inserção tabela {cod}")
        df_codra = client.persist(df[df['cod_ra']==cod])
        df_codra.to_sql(f"tb_codra_dask_{cod}",url_db, if_exists='replace', index=False, compute = True, parallel = True, chunksize=500,dtype={
            k.lower(): p.oracle_types[v] for (k,v) in p.schema_base.items()
            })
        end_time = time.time()
        elapsed_time = end_time - start_time
        logger.success(f"Inserção da tabela {cod} concluida,{elapsed_time} ")
    except:
        logger.error("Erro na task FiltraCodRa")

async def escreve_tabelas():
    tasks = [asyncio.to_thread(filtra_cod, cod) for cod in df.cod_ra.unique()]
    res = await asyncio.gather(*tasks)

In [135]:
#for i in range(7):
df=read_oracle(meta=p.meta_schema)
df=client.persist(df)    
await escreve_tabelas()

Pandera 
=

In [ ]:
schema= pa.DataFrameSchema({
    '00AAAJ': pa.Column(str,checks=pa.Check.isin(['aaaa']))
})

In [ ]:
df = dd.read_sql_table(table_name='tb_raw_data', con=url_db,index_col= 'cod', meta= meta_schema)
df=client.persist(df)

In [ ]:
try:
    schema.validate(df.compute())
except pa.errors.SchemaError as err: 
    df_erros= err.failure_cases
    

In [ ]:
df_erros.info